In [113]:
import yfinance as yf
from datetime import datetime, timedelta
import pandas as pd
import requests
import QuantLib as ql
import numpy as np
import matplotlib.pyplot as plt
import json
from sklearn.metrics import mean_squared_error
import scipy.optimize as opt
from scipy.optimize import differential_evolution





with open("config.json", "r") as config_file:
    config = json.load(config_file)

api_key = config.get("api_key")
secret_key = config.get("secret_key")


In [32]:
NASDAQ = pd.read_csv('Indexes/NASDAQ.csv')
DOWJ = pd.read_csv('Indexes/DOWJ.csv')
SP = pd.read_csv('Indexes/S&P500.csv')

def clean_data(df):
    df = df[['Company', 'Symbol']]
    df = pd.DataFrame(df).dropna()
    return df

NASDAQ = clean_data(NASDAQ)
DOWJ = clean_data(DOWJ)
SP = clean_data(SP)


In [133]:
    
# SCREENS STOCKS WITH 1-DAY CHANGE OF -2% AND ROLLING AVG OF CHANGE OVER 2 WEEKS OF > 0%
def filter_stocks(): 
    filtered_stocks = []
    for index, stock in NASDAQ.iterrows():
        today_change, rolling_avg = get_price_change_and_rolling_avg(stock['Symbol'])
        if (today_change <= -2.00) and (rolling_avg > 0.00): 
            filtered_stocks.append(stock['Symbol'])

    for index, stock in DOWJ.iterrows():
        today_change, rolling_avg = get_price_change_and_rolling_avg(stock['Symbol'])
        if (today_change <= -2.00) and (rolling_avg > 0.00): 
            filtered_stocks.append(stock['Symbol'])

    for index, stock in NASDAQ.iterrows():
        today_change, rolling_avg = get_price_change_and_rolling_avg(stock['Symbol'])
        if (today_change <= -2.00) and (rolling_avg > 0.00): 
            filtered_stocks.append(stock['Symbol'])

    return filter_stocks


def get_price_change_and_rolling_avg(ticker: str):

    end_date = datetime.now()
    start_date = end_date - timedelta(days=24)
    
    data = yf.download(ticker, start=start_date, end=end_date, progress=False)
    
    data = data.sort_index()
    
    data['Price_Change'] = (data['Close'].diff() / data['Close'].shift(1)) * 100
    today_price_change = data['Price_Change'].iloc[-1]

    rolling_avg = data['Price_Change'].rolling(window=min(14, len(data))).mean().iloc[-1]
    
    return today_price_change, rolling_avg

def get_current_stock_price(symbol: str):

    url = "https://data.alpaca.markets/v2/stocks/trades/latest"

    headers = {
        "accept": "application/json",
        "APCA-API-KEY-ID": api_key,
        "APCA-API-SECRET-KEY": secret_key,
    }

    params = {
        "symbols": symbol,  
        "feed": "iex" 
    }

    try:
        response = requests.get(url, headers=headers, params=params)
        response.raise_for_status()  

        data = response.json()
        return data.get("trades", {}).get(symbol, {}).get("p") 

    except requests.exceptions.RequestException as e:
        print(f"Error fetching stock price: {e}")


def fetch_option_chain(api_key: str, secret_key: str, underlying_symbol: str):

    url = f"https://data.alpaca.markets/v1beta1/options/snapshots/{underlying_symbol}?feed=indicative&limit=100&expiration_date=2025-02-07"
    headers = {
        "accept": "application/json",
        "APCA-API-KEY-ID": api_key,
        "APCA-API-SECRET-KEY": secret_key,
    }
    try:
        response = requests.get(url, headers=headers)
        data = response.json()
        snapshots = data.get('snapshots', {})
        return snapshots
    except requests.exceptions.RequestException as e:
        print(f"Error fetching option chain: {e}")
        return None

def parse_option_chain(option_chain):

    parsed_data = []
    
    for symbol, details in option_chain.items():
        expiration_start = len(symbol) - 15
        ticker = symbol[:expiration_start]
        expiration_date = f"20{symbol[expiration_start:expiration_start+2]}-{symbol[expiration_start+2:expiration_start+4]}-{symbol[expiration_start+4:expiration_start+6]}"
        option_type = "Call" if symbol[expiration_start+6] == "C" else "Put"
        strike_price = int(symbol[expiration_start+7:]) / 1000  
        
        greeks = details.get("greeks", {}) or {}
        implied_volatility = details.get("impliedVolatility", None)
        latest_quote = details.get("latestQuote", {})
        
        parsed_data.append({
            "ticker": ticker,
            "expiration_date": expiration_date,
            "option_type": option_type,
            "strike_price": strike_price,
            "delta": greeks.get("delta"),
            "gamma": greeks.get("gamma"),
            "rho": greeks.get("rho"),
            "theta": greeks.get("theta"),
            "vega": greeks.get("vega"),
            "implied_volatility": implied_volatility,
            "ask_price": latest_quote.get("ap"),
            "ask_size": latest_quote.get("as"),
            "bid_price": latest_quote.get("bp"),
            "bid_size": latest_quote.get("bs")
        })
    
    return pd.DataFrame(parsed_data)

def get_option_chain(api_key: str, secret_key: str, underlying_symbol: str):

    option_chain = fetch_option_chain(api_key, secret_key, underlying_symbol)
    if option_chain:
        return parse_option_chain(option_chain)
    else:
        return pd.DataFrame()
    

def gbm(s0, mu, sigma, deltaT, dt):
    """
    Models a stock price S(t) using the Wiener process W(t) as
    `S(t) = S(0).exp{(mu-(sigma^2/2).t)+sigma.W(t)}`
    
    Arguments:
        s0: Initial stock price, default 100
        mu: 'Drift' of the stock (upwards or downwards), default 0.2
        sigma: 'Volatility' of the stock, default 0.68
        deltaT: The time period for which the future prices are computed, default 52 (as in 52 weeks)
        dt: The granularity of the time-period, default 0.1
    
    Returns:
        time_vector: array of time steps
        s: array with the simulated stock prices over the time-period deltaT
    """
    n_step = int(deltaT / dt)  # Number of time steps
    time_vector = np.linspace(0, deltaT, num=n_step)  # Time vector
    
    # Wiener process: cumulative sum of random normal increments
    random_increments = np.random.normal(0, np.sqrt(dt), size=n_step)
    weiner_process = np.cumsum(random_increments)
    
    # Stock price simulation
    stock_var = (mu - (sigma**2 / 2)) * time_vector
    s = s0 * np.exp(stock_var + sigma * weiner_process)
    
    return s



def objective(params, real_prices, s0):
    """Objective function for optimization."""
    mu, sigma = params  # Unpack parameters
    gbm_prices = gbm(s0, mu, sigma, deltaT=len(real_prices), dt=1)
    return mean_squared_error(real_prices, gbm_prices)

def optimize_gbm(symbol):
    """
    Optimize μ and σ over multiple time bins, weighting recent periods more.
    """
    # Fetch real stock data (past 5 years)
    stock_data = yf.download(symbol, period="5y", interval="1d")
    real_prices = stock_data["Adj Close"].dropna().values

    # Split into bins of 20 trading days
    bin_size = 20
    num_bins = len(real_prices) // bin_size
    weights = np.linspace(1, 2, num_bins)  # Increasing weights for recent bins

    mu_values, sigma_values, mses = [], [], []

    # Optimize each bin
    for i in range(num_bins):
        bin_prices = real_prices[i * bin_size : (i + 1) * bin_size]
        s0 = bin_prices[0]

        # Define the bounds for optimization
        bounds = [(-0.3, 0.3), (0.01, 0.35)]

        # Run the optimizer
        result = differential_evolution(objective, bounds, args=(bin_prices, s0))
        best_mu, best_sigma = result.x
        best_mse = result.fun

        mu_values.append(best_mu)
        sigma_values.append(best_sigma)
        mses.append(best_mse)

        print(f"Bin {i+1}/{num_bins}: μ = {best_mu:.4f}, σ = {best_sigma:.4f}, MSE = {best_mse:.4f}")

    # Compute weighted averages
    weight_sum = np.sum(weights)
    avg_mu = np.sum(np.array(mu_values) * weights) / weight_sum
    avg_sigma = np.sum(np.array(sigma_values) * weights) / weight_sum

    print(f"\nFinal Weighted Averages: μ = {avg_mu:.4f}, σ = {avg_sigma:.4f}")

    return avg_mu, avg_sigma

# Example usage
avg_mu, avg_sigma = optimize_gbm("AAPL")

[*********************100%***********************]  1 of 1 completed


Bin 1/62: μ = 0.0016, σ = 0.0259, MSE = 3.1037
Bin 2/62: μ = -0.0100, σ = 0.0185, MSE = 4.9194
Bin 3/62: μ = 0.0079, σ = 0.0168, MSE = 1.6974
Bin 4/62: μ = 0.0053, σ = 0.0108, MSE = 0.2305
Bin 5/62: μ = 0.0074, σ = 0.0113, MSE = 0.9216
Bin 6/62: μ = 0.0027, σ = 0.0125, MSE = 1.0844
Bin 7/62: μ = 0.0153, σ = 0.0263, MSE = 3.8795
Bin 8/62: μ = -0.0044, σ = 0.0188, MSE = 7.3880
Bin 9/62: μ = 0.0023, σ = 0.0139, MSE = 2.4322
Bin 10/62: μ = -0.0003, σ = 0.0158, MSE = 2.8260
Bin 11/62: μ = 0.0004, σ = 0.0133, MSE = 1.8790
Bin 12/62: μ = 0.0029, σ = 0.0111, MSE = 2.4123
Bin 13/62: μ = 0.0091, σ = 0.0173, MSE = 3.7889
Bin 14/62: μ = -0.0066, σ = 0.0124, MSE = 2.7793
Bin 15/62: μ = 0.0036, σ = 0.0150, MSE = 2.1095
Bin 16/62: μ = -0.0009, σ = 0.0104, MSE = 1.2119
Bin 17/62: μ = -0.0032, σ = 0.0111, MSE = 1.3344
Bin 18/62: μ = 0.0079, σ = 0.0101, MSE = 0.9314
Bin 19/62: μ = -0.0009, σ = 0.0112, MSE = 1.3987
Bin 20/62: μ = 0.0040, σ = 0.0110, MSE = 1.3390
Bin 21/62: μ = -0.0011, σ = 0.0106, MSE = 

In [137]:
candidates = filter_stocks()

option_chain = get_option_chain(api_key, secret_key, 'AAPL')

put_chain = option_chain[(option_chain['option_type'] == 'Put') & (option_chain['rho'].notna())].sort_values(by='strike_price', ascending=True)
price = get_current_stock_price('AAPL')



# Initialize lists to store results
profitability_chances = []
percent_returns = []

# Loop through each put option in the chain and simulate price movements
for index, contract in put_chain.iterrows():
    count = 0
    strike_price = contract['strike_price']
    
    # Simulate price movements 100 times
    for i in range(100):
        prices = gbm(s0=price, mu=0.0011, sigma=0.0130, deltaT=20, dt=1)
        
        if prices[-1] > strike_price:
            count += 1
    
    # Calculate profitability chance and percent return
    profitability_chance = (count / 100) * 100
    profit = (contract['bid_price']*contract['bid_size'] + contract['ask_price']*contract['ask_size']) / (contract['ask_size'] + contract['bid_size'])
    percent_return = profit / (strike_price * 100) * 100
    
    # Append results
    profitability_chances.append(profitability_chance)
    percent_returns.append(percent_return)

# Add the results as new columns in the DataFrame
put_chain['profitability_percent'] = profitability_chances
put_chain['percent_return'] = percent_returns

# Display the updated DataFrame
put_chain


,ticker,expiration_date,option_type,strike_price,delta,gamma,rho,theta,vega,implied_volatility,ask_price,ask_size,bid_price,bid_size,profitability_percent,percent_return
0,AAPL,2025-02-07,Put,187.5,-0.0085,0.0010,-0.0003,-0.0512,0.0064,0.8106,0.10,197,0.03,137,100.0,0.000380
5,AAPL,2025-02-07,Put,190.0,-0.0090,0.0012,-0.0003,-0.0507,0.0066,0.7675,0.09,288,0.04,100,100.0,0.000406
20,AAPL,2025-02-07,Put,192.5,-0.0083,0.0012,-0.0003,-0.0438,0.0062,0.7095,0.07,1,0.04,13,100.0,0.000219
39,AAPL,2025-02-07,Put,195.0,-0.0061,0.0010,-0.0002,-0.0298,0.0047,0.6322,0.06,1,0.01,28,100.0,0.000060
50,AAPL,2025-02-07,Put,197.5,-0.0093,0.0015,-0.0003,-0.0427,0.0069,0.6265,0.09,168,0.02,203,100.0,0.000262
33,AAPL,2025-02-07,Put,200.0,-0.0145,0.0021,-0.0005,-0.0627,0.0101,0.6259,0.11,189,0.07,219,100.0,0.000443
7,AAPL,2025-02-07,Put,202.5,-0.0238,0.0032,-0.0008,-0.0972,0.0154,0.6368,0.29,80,0.03,73,100.0,0.000819
25,AAPL,2025-02-07,Put,205.0,-0.0159,0.0027,-0.0005,-0.0579,0.0109,0.5354,0.12,131,0.05,230,100.0,0.000368
4,AAPL,2025-02-07,Put,207.5,-0.0164,0.0031,-0.0005,-0.0542,0.0112,0.4885,0.09,61,0.07,133,99.0,0.000368
38,AAPL,2025-02-07,Put,210.0,-0.0262,0.0046,-0.0009,-0.0800,0.0167,0.4849,0.14,1,0.13,92,99.0,0.000620


In [67]:
# TODO
"""
- Figure out way to normalize stock price, whether that is min max of the range of the price(shoudl help optimizer
- Find better metric for optimizer
- Try binning, so like get past 10 years of AAPL, seperate into bins of 20 or n trading days, train optimzer on each one. Then the hyperparameters can be weighted to have more bias towards more recent bins
"""


[*********************100%***********************]  1 of 1 completed


20